In [19]:
import torch
import torchaudio
from torchaudio.models._wavernn import _WaveRNN
from collections import OrderedDict

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

checkpoint_path = './model_best.pth.tar'
checkpoint = torch.load(checkpoint_path, map_location=device)

state_dict = checkpoint["state_dict"]

model = _WaveRNN(
    n_bits=8,
    upsample_scales=[5, 5, 11],
    sample_rate=22050,
    hop_length=275,
    n_freq=80,
)

# model.load_state_dict(state_dict) has error because the model is saved using `nn.DataParallel`
# https://discuss.pytorch.org/t/solved-keyerror-unexpected-key-module-encoder-embedding-weight-in-state-dict/1686
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    # remove `module.`
    name = k[7:]
    new_state_dict[name] = v

# model.load_state_dict(new_state_dict)

save_path = './model_wavernn.pth'
torch.save(new_state_dict, save_path)

model.load_state_dict(torch.load(save_path, map_location=device))

<All keys matched successfully>